 # 一个完整GPT2训练流程

## 定义tokenizer和文本处理

>  没办法，使用autodl启用学术加速，不需要的不执行就好


In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [9]:
from transformers import GPT2Tokenizer
#
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
def preprocess(text):
    encoding = tokenizer.encode_plus(text,max_length=512 ,truncation=True, padding='max_length', return_tensors='pt')
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask


### 测试显卡命令

In [ ]:
!nvidia-smi

In [9]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [10]:
!gpustat -c -p -u

autodl-container-28ce11b552-1f3ab94c  Fri Sep 15 18:00:01 2023  525.89.02
[0] NVIDIA GeForce RTX 2080 Ti | 31°C,   0 % |     0 / 11264 MB |


# train函数的定义

In [11]:
#train函数的定义
import torch

def train(model, optimizer, epochs, training_loader):
    model.to(device)
    model.train()
    save_path = './model/'
    for epoch in range(epochs):
        loss = 0
        for i, batch in enumerate(training_loader):
            inputs, targets = batch
#             print(inputs.shape,targets.shape)
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            if (i+1) % 20== 0:
                print('Epoch {:2d} | Batch {:2d}/{:2d} | Loss {:.4f}'.format(epoch+1, i+1, len(training_loader), loss))
        torch.save(model.state_dict(), save_path + 'epoch{}.pt'.format(epoch+1))#设置保存路径
        # torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pt')
print('finish')

finish


# 加载数据集

In [13]:

from torch.utils.data import Dataset, DataLoader
import pandas as pd
#定义数据类
class NewsDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_ids, attention_mask = preprocess(self.data.iloc[index]['description'])
        input_ids = torch.tensor(input_ids).squeeze()
        attention_mask = torch.tensor(attention_mask).squeeze()
        return input_ids, attention_mask

dataset = pd.read_csv('./bbc_news.csv')[['description']]

training_set = NewsDataset(dataset[:5000])
training_loader = DataLoader(training_set, batch_size=4)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device='cpu'
print('finish setting')

finish setting


# 训练命令

In [14]:
from transformers import AdamW
from transformers import GPT2LMHeadModel
model=GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [15]:
from transformers import AdamW
from transformers import GPT2LMHeadModel
model=GPT2LMHeadModel.from_pretrained('gpt2').to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
train(model, optimizer, 2, training_loader)
model

/root/miniconda3/envs/mocheg/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_4352/2276305946.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).squeeze()
/tmp/ipykernel_4352/2276305946.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask).squeeze()


Epoch  1 | Batch 20/1250 | Loss 0.5551
Epoch  1 | Batch 40/1250 | Loss 0.4463
Epoch  1 | Batch 60/1250 | Loss 0.3926
Epoch  1 | Batch 80/1250 | Loss 0.1702
Epoch  1 | Batch 100/1250 | Loss 0.0983
Epoch  1 | Batch 120/1250 | Loss 0.0693
Epoch  1 | Batch 140/1250 | Loss 0.0427
Epoch  1 | Batch 160/1250 | Loss 0.0437
Epoch  1 | Batch 180/1250 | Loss 0.0237
Epoch  1 | Batch 200/1250 | Loss 0.0165
Epoch  1 | Batch 220/1250 | Loss 0.0201
Epoch  1 | Batch 240/1250 | Loss 0.0137
Epoch  1 | Batch 260/1250 | Loss 0.0098
Epoch  1 | Batch 280/1250 | Loss 0.0131
Epoch  1 | Batch 300/1250 | Loss 0.0122
Epoch  1 | Batch 320/1250 | Loss 0.0174
Epoch  1 | Batch 340/1250 | Loss 0.0090
Epoch  1 | Batch 360/1250 | Loss 0.0118
Epoch  1 | Batch 380/1250 | Loss 0.0045
Epoch  1 | Batch 400/1250 | Loss 0.0062
Epoch  1 | Batch 420/1250 | Loss 0.0029
Epoch  1 | Batch 440/1250 | Loss 0.0048
Epoch  1 | Batch 460/1250 | Loss 0.0051
Epoch  1 | Batch 480/1250 | Loss 0.0026
Epoch  1 | Batch 500/1250 | Loss 0.0088
Epoc

KeyboardInterrupt: 

# 测试脚本

In [14]:
text_input=torch.tensor(tokenizer.encode("trump ", add_special_tokens=True))
text_input=text_input.to(device)
generated = model.generate(
    input_ids = text_input.unsqueeze(0),
    max_length = 50,
    temperature = 1.0,
    top_k = 0,
    top_p = 0.9,
    do_sample=True,
    num_return_sequences=1
)
print(text_input)
generated_title = tokenizer.decode(generated[0], skip_special_tokens=True)
print(generated_title)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([40954,   220], device='cuda:0')
trump!!!";!!!!";";";";""";"""""""""""""""""""""""""""""


In [ ]:
!pip list

In [ ]:
!nvidia-smi


## 对原生的gpt2进行测试

In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
config = GPT2Config.from_pretrained('gpt2')
# model = GPT2LMHeadModel(config)
# model_path = './checkpoint_epoch_1.pt'
# model_state_dict = torch.load(model_path)
# model.load_state_dict(model_state_dict)
model = GPT2LMHeadModel.from_pretrained('gpt2')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)
text_input=torch.tensor(tokenizer.encode("trump is president of United States,  ", add_special_tokens=True))
text_input=text_input.to(device)
generated = model.generate(
    input_ids = text_input.unsqueeze(0),
    max_length = 50,
    temperature = 1.0,
    top_k = 0,
    top_p = 0.9,
    do_sample=True,
    num_return_sequences=1
)
print(text_input)
generated_title = tokenizer.decode(generated[0], skip_special_tokens=True)
print(generated_title)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([40954,   318,  1893,   286,  1578,  1829,    11,   220,   220],
       device='cuda:0')
trump is president of United States,         ------------ Reply--

Thread


Link

He chose to defend Donald Trump over a "stupid y'all."


What a y'all.
